In [1]:
import torch
import torch.nn as nn

In [2]:
import Encoder as E
import Decoder as D

x = torch.randn(16,3,28,28,device = 'cuda')
model1 = E.ENCODER(input_dims=3,hidden_dims=128).to(device = 'cuda')
model2 = D.DECODER(output_dims=3,hidden_dims=128).to(device = 'cuda')

print(x.shape)
print(model1(x).shape) 
print(model2(model1(x)).shape)


c:\Users\kaasa\AppData\Local\Programs\Python\Python310\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")



torch.Size([16, 3, 28, 28])
torch.Size([16, 128, 6, 6])
torch.Size([16, 3, 28, 28])


In [4]:

from typing import Any
import lightning.pytorch as pl
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
import torch.nn.functional as F

training_dataset = CIFAR10(root='./data',train=True,download=True,transform=transforms.Compose([
            transforms.RandomResizedCrop(28),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor()]))
validation_dataset = CIFAR10(root='./data',train=False,download=True,transform=transforms.Compose([
            transforms.RandomResizedCrop(28),
            transforms.ToTensor()]))

training_dataloader = DataLoader(training_dataset,batch_size=128)
validation_dataloader = DataLoader(validation_dataset,batch_size=128)

class AUTOENCODER(pl.LightningModule):
    def __init__(self, 
                 input_dim = 3,
                 hiddern_dim =128,):
        super().__init__()
        self.encoder = E.ENCODER(input_dims=input_dim,hidden_dims=hiddern_dim)
        self.decoder = D.DECODER(output_dims=input_dim,hidden_dims=hiddern_dim)
        self.automatic_optimization=False

    def forward(self,x):
        z_e = self.encoder(x)
        out = self.decoder(z_e)

        return out ,x , z_e
    
    def training_step(self, batch, batch_idx):
        opt = self.optimizers()

        X, _ = batch
        d_x = self(X)
        loss = F.mse_loss(X, d_x, reduction='none')
        opt.zero_grad()
        self.manual_backward(loss)
        opt.step()

        self.log_dict({"recon_loss": loss}, prog_bar=True)

    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(), lr=1e-5)
        return opt 
    
    
model = AUTOENCODER(3,64)

trainer = pl.Trainer(max_epochs=5, devices=1)

trainer.fit(model,train_dataloaders=training_dataloader,
                    val_dataloaders=validation_dataloader)


Files already downloaded and verified
Files already downloaded and verified


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type    | Params
------------------------------------
0 | encoder | ENCODER | 23.3 K
1 | decoder | DECODER | 101 K 
------------------------------------
124 K     Trainable params
0         Non-trainable params
124 K     Total params
0.498     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

AttributeError: 'tuple' object has no attribute 'size'